In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler


df = pd.read_csv(r"C:\Users\jjap2\Poleto SCE\Poleto-SCE\dond_game_data.csv")

In [2]:
#cleaning cell
df["Remaining Values"] = df["Remaining Values"].apply(ast.literal_eval)

all_values = set()
for values in df["Remaining Values"]:
    all_values.update(values)

for value in sorted(all_values):
    df[f"Case_{value}"] = df["Remaining Values"].apply(lambda x: 1 if value in x else 0)

df.drop(columns=["Remaining Values"], inplace=True)
df.drop(columns=["Contestant Name"], inplace=True)

categorical_columns = ["Contestant Gender", "Contestant Race"]
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

In [3]:
# defining variables
X = df.drop(columns=["Offer"])
y = df["Offer"]

In [4]:
# standardize!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

lasso = LassoCV(cv=5, random_state=42).fit(X_train, y_train)

In [7]:
Alpha = lasso.alpha_
lasso_coefs = lasso.coef_

print("Alpha:", Alpha)
print("LASSO Coefficients:", lasso_coefs)

Alpha: 102.35749552417937
LASSO Coefficients: [-1.64682964e+01  9.25925245e+03 -2.81935066e+04  1.24093999e+05
 -4.61560432e+02  3.43371398e+03  3.20291622e+04  1.95378778e+03
 -1.29902898e+03  6.18709701e+03 -4.37207744e+03  2.31555314e+03
  3.32703615e+03  7.59231924e+02  1.89922120e+03  3.74992759e+03
 -0.00000000e+00  3.62721545e+03  3.15759662e+03  2.91698604e+03
  2.11013909e+03  3.27692660e+03  2.88449604e+03  2.09962166e+03
  4.57737307e+03  9.43586219e+02  0.00000000e+00  2.86615936e+03
 -1.19780428e+03 -0.00000000e+00  5.25969740e+03  2.29499587e+03
 -0.00000000e+00  2.39394348e+03  1.41931183e+03  9.76661258e+01
  4.49003848e+03  1.26837042e+03  2.45955141e+03 -7.15417391e+03
  1.66426285e+03  0.00000000e+00  1.22700554e+03 -1.70915424e+03
 -5.44142464e+03 -7.18134007e+02 -6.42080027e+03 -4.68761375e+02
 -3.20557561e+03 -0.00000000e+00 -9.54883889e+03 -2.62084743e+02
  5.39106716e+02  0.00000000e+00 -2.80022205e+03  2.09476360e+02]
